<a href="https://colab.research.google.com/github/camylaand/mapeamento-de-comportamento-docs/blob/main/Projeto(oficial).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import os
from sklearn.preprocessing import OneHotEncoder
import joblib
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.cluster import KMeans
from scipy.stats import f_oneway
import statsmodels.formula.api as smf

In [2]:
# Fixar a semente para reprodutibilidade
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

In [3]:
df = pd.read_csv('/content/transacoes_final_fraude.csv')
df

,_id,transacao_id,cliente_id,conta_id,conta_destino_id,mesma_titularidade,transacao_data,transacao_valor,transacao_tipo
0,67b377336d1479f56b001221,8c6a2055,356,1cbe0f3f1,10a4b8c06,False,2023-01-01 00:00:30,2474.50,pix
1,67b377336d1479f56b001222,c2970a80,596,1bf29835c,113db0daf,False,2023-01-01 00:00:31,755.14,pix
2,67b377336d1479f56b001223,6d9575d5,179,1fd5f554a,1fd5f554a,True,2023-01-01 00:00:33,3539.75,saque
3,67b377336d1479f56b001224,507c00db,53,1f2c8a8d7,1c8745726,False,2023-01-01 00:03:12,338.06,pix
4,67b377336d1479f56b001225,26377a88,449,1053de403,1053de403,True,2023-01-01 00:05:22,3936.60,saque
...,...,...,...,...,...,...,...,...,...
1764563,67b377b26d1479f56b1afef4,910e20ab,552,1db52b9e5,2970fe019,True,2023-12-31 23:57:57,3325.66,deposito
1764564,67b377b26d1479f56b1afef5,de85de8f,296,1fe6878a3,22991322d,False,2023-12-31 23:58:31,6696.30,pagamento
1764565,67b377b26d1479f56b1afef6,df469b5e,217,103da94f1,1936f5d4c,False,2023-12-31 23:59:14,4510.69,pix
1764566,67b377b26d1479f56b1afef7,6912b7e4,26,10d888039,10d888039,True,2023-12-31 23:59:29,3253.94,saque


In [4]:
#Observar possíveis irregularidades
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1764568 entries, 0 to 1764567
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   _id                 object 
 1   transacao_id        object 
 2   cliente_id          int64  
 3   conta_id            object 
 4   conta_destino_id    object 
 5   mesma_titularidade  bool   
 6   transacao_data      object 
 7   transacao_valor     float64
 8   transacao_tipo      object 
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 109.4+ MB


In [5]:
#Verificando tipos de dados
df['transacao_tipo'].unique()

array(['pix', 'saque', 'transferencia', 'pagamento', 'deposito'],
      dtype=object)

In [6]:
# Converter a coluna 'transacao_data' para o formato datetime
df['transacao_data'] = pd.to_datetime(df['transacao_data'])
df

,_id,transacao_id,cliente_id,conta_id,conta_destino_id,mesma_titularidade,transacao_data,transacao_valor,transacao_tipo
0,67b377336d1479f56b001221,8c6a2055,356,1cbe0f3f1,10a4b8c06,False,2023-01-01 00:00:30,2474.50,pix
1,67b377336d1479f56b001222,c2970a80,596,1bf29835c,113db0daf,False,2023-01-01 00:00:31,755.14,pix
2,67b377336d1479f56b001223,6d9575d5,179,1fd5f554a,1fd5f554a,True,2023-01-01 00:00:33,3539.75,saque
3,67b377336d1479f56b001224,507c00db,53,1f2c8a8d7,1c8745726,False,2023-01-01 00:03:12,338.06,pix
4,67b377336d1479f56b001225,26377a88,449,1053de403,1053de403,True,2023-01-01 00:05:22,3936.60,saque
...,...,...,...,...,...,...,...,...,...
1764563,67b377b26d1479f56b1afef4,910e20ab,552,1db52b9e5,2970fe019,True,2023-12-31 23:57:57,3325.66,deposito
1764564,67b377b26d1479f56b1afef5,de85de8f,296,1fe6878a3,22991322d,False,2023-12-31 23:58:31,6696.30,pagamento
1764565,67b377b26d1479f56b1afef6,df469b5e,217,103da94f1,1936f5d4c,False,2023-12-31 23:59:14,4510.69,pix
1764566,67b377b26d1479f56b1afef7,6912b7e4,26,10d888039,10d888039,True,2023-12-31 23:59:29,3253.94,saque


In [7]:
dias_semana = {0:'Segunda', 1:'Terca', 2:'Quarta', 3:'Quinta', 4:'Sexta', 5:'Sabado', 6:'Domingo'}
df['dia_de_semana'] = df['transacao_data'].dt.weekday.map(dias_semana)
df['fim_de_semana'] = df['dia_de_semana'].apply(lambda x: 1 if x in ['Sabado', 'Domingo'] else 0)
df

,_id,transacao_id,cliente_id,conta_id,conta_destino_id,mesma_titularidade,transacao_data,transacao_valor,transacao_tipo,dia_de_semana,fim_de_semana
0,67b377336d1479f56b001221,8c6a2055,356,1cbe0f3f1,10a4b8c06,False,2023-01-01 00:00:30,2474.50,pix,Domingo,1
1,67b377336d1479f56b001222,c2970a80,596,1bf29835c,113db0daf,False,2023-01-01 00:00:31,755.14,pix,Domingo,1
2,67b377336d1479f56b001223,6d9575d5,179,1fd5f554a,1fd5f554a,True,2023-01-01 00:00:33,3539.75,saque,Domingo,1
3,67b377336d1479f56b001224,507c00db,53,1f2c8a8d7,1c8745726,False,2023-01-01 00:03:12,338.06,pix,Domingo,1
4,67b377336d1479f56b001225,26377a88,449,1053de403,1053de403,True,2023-01-01 00:05:22,3936.60,saque,Domingo,1
...,...,...,...,...,...,...,...,...,...,...,...
1764563,67b377b26d1479f56b1afef4,910e20ab,552,1db52b9e5,2970fe019,True,2023-12-31 23:57:57,3325.66,deposito,Domingo,1
1764564,67b377b26d1479f56b1afef5,de85de8f,296,1fe6878a3,22991322d,False,2023-12-31 23:58:31,6696.30,pagamento,Domingo,1
1764565,67b377b26d1479f56b1afef6,df469b5e,217,103da94f1,1936f5d4c,False,2023-12-31 23:59:14,4510.69,pix,Domingo,1
1764566,67b377b26d1479f56b1afef7,6912b7e4,26,10d888039,10d888039,True,2023-12-31 23:59:29,3253.94,saque,Domingo,1


In [8]:
def categorizar_hora(hora):
  hora = hora.hour
  if hora < 6:
    return 'Madrugada'
  elif hora < 12:
    return 'Manhã'
  elif hora < 18:
    return 'Tarde'
  else:
    return 'Noite'
df['faixa_horaria'] = df['transacao_data'].dt.time.apply(categorizar_hora)

In [9]:
#Remover a coluna 'transacao_data', pois não será usada no modelo
df = df.drop('transacao_data', axis = 1)
df

,_id,transacao_id,cliente_id,conta_id,conta_destino_id,mesma_titularidade,transacao_valor,transacao_tipo,dia_de_semana,fim_de_semana,faixa_horaria
0,67b377336d1479f56b001221,8c6a2055,356,1cbe0f3f1,10a4b8c06,False,2474.50,pix,Domingo,1,Madrugada
1,67b377336d1479f56b001222,c2970a80,596,1bf29835c,113db0daf,False,755.14,pix,Domingo,1,Madrugada
2,67b377336d1479f56b001223,6d9575d5,179,1fd5f554a,1fd5f554a,True,3539.75,saque,Domingo,1,Madrugada
3,67b377336d1479f56b001224,507c00db,53,1f2c8a8d7,1c8745726,False,338.06,pix,Domingo,1,Madrugada
4,67b377336d1479f56b001225,26377a88,449,1053de403,1053de403,True,3936.60,saque,Domingo,1,Madrugada
...,...,...,...,...,...,...,...,...,...,...,...
1764563,67b377b26d1479f56b1afef4,910e20ab,552,1db52b9e5,2970fe019,True,3325.66,deposito,Domingo,1,Noite
1764564,67b377b26d1479f56b1afef5,de85de8f,296,1fe6878a3,22991322d,False,6696.30,pagamento,Domingo,1,Noite
1764565,67b377b26d1479f56b1afef6,df469b5e,217,103da94f1,1936f5d4c,False,4510.69,pix,Domingo,1,Noite
1764566,67b377b26d1479f56b1afef7,6912b7e4,26,10d888039,10d888039,True,3253.94,saque,Domingo,1,Noite


In [10]:
df = df.drop(columns=['transacao_id', 'conta_destino_id'])
df

,_id,cliente_id,conta_id,mesma_titularidade,transacao_valor,transacao_tipo,dia_de_semana,fim_de_semana,faixa_horaria
0,67b377336d1479f56b001221,356,1cbe0f3f1,False,2474.50,pix,Domingo,1,Madrugada
1,67b377336d1479f56b001222,596,1bf29835c,False,755.14,pix,Domingo,1,Madrugada
2,67b377336d1479f56b001223,179,1fd5f554a,True,3539.75,saque,Domingo,1,Madrugada
3,67b377336d1479f56b001224,53,1f2c8a8d7,False,338.06,pix,Domingo,1,Madrugada
4,67b377336d1479f56b001225,449,1053de403,True,3936.60,saque,Domingo,1,Madrugada
...,...,...,...,...,...,...,...,...,...
1764563,67b377b26d1479f56b1afef4,552,1db52b9e5,True,3325.66,deposito,Domingo,1,Noite
1764564,67b377b26d1479f56b1afef5,296,1fe6878a3,False,6696.30,pagamento,Domingo,1,Noite
1764565,67b377b26d1479f56b1afef6,217,103da94f1,False,4510.69,pix,Domingo,1,Noite
1764566,67b377b26d1479f56b1afef7,26,10d888039,True,3253.94,saque,Domingo,1,Noite


In [11]:
df['mesma_titularidade'] = df['mesma_titularidade'].astype(int)
df

,_id,cliente_id,conta_id,mesma_titularidade,transacao_valor,transacao_tipo,dia_de_semana,fim_de_semana,faixa_horaria
0,67b377336d1479f56b001221,356,1cbe0f3f1,0,2474.50,pix,Domingo,1,Madrugada
1,67b377336d1479f56b001222,596,1bf29835c,0,755.14,pix,Domingo,1,Madrugada
2,67b377336d1479f56b001223,179,1fd5f554a,1,3539.75,saque,Domingo,1,Madrugada
3,67b377336d1479f56b001224,53,1f2c8a8d7,0,338.06,pix,Domingo,1,Madrugada
4,67b377336d1479f56b001225,449,1053de403,1,3936.60,saque,Domingo,1,Madrugada
...,...,...,...,...,...,...,...,...,...
1764563,67b377b26d1479f56b1afef4,552,1db52b9e5,1,3325.66,deposito,Domingo,1,Noite
1764564,67b377b26d1479f56b1afef5,296,1fe6878a3,0,6696.30,pagamento,Domingo,1,Noite
1764565,67b377b26d1479f56b1afef6,217,103da94f1,0,4510.69,pix,Domingo,1,Noite
1764566,67b377b26d1479f56b1afef7,26,10d888039,1,3253.94,saque,Domingo,1,Noite


In [12]:
#Criando instância OneHotEncoder para codificar categorias
encoder_tipo_transacao = OneHotEncoder(categories=[['pix','saque','transferencia','pagamento','deposito']], sparse_output=False)
df_tipo = pd.DataFrame(encoder_tipo_transacao.fit_transform(df[['transacao_tipo']]), columns=encoder_tipo_transacao.get_feature_names_out(['transacao_tipo']))

In [13]:
encoder_semana = OneHotEncoder(categories = [['Segunda', 'Terca', 'Quarta', 'Quinta', 'Sexta', 'Sabado', 'Domingo']], sparse_output = False)
df_semana = pd.DataFrame(encoder_semana.fit_transform(df[['dia_de_semana']]), columns=encoder_semana.get_feature_names_out(['dia_de_semana']))

In [14]:
encoder_horario = OneHotEncoder(categories = [['Madrugada', 'Manhã', 'Tarde', 'Noite']], sparse_output = False)
df_hora = pd.DataFrame(encoder_horario.fit_transform(df[['faixa_horaria']]), columns=encoder_horario.get_feature_names_out(['faixa_horaria']))

In [15]:
dados = pd.concat([df.drop(columns=['transacao_tipo', 'dia_de_semana', 'faixa_horaria']), df_tipo, df_semana, df_hora], axis=1)

In [16]:
#Salvar e carregar modelos de Machine Learning, processar dados em paralelo e otimizar operações pesadas
joblib.dump(encoder_tipo_transacao, 'encoder_tipo_transacao.pkl')
joblib.dump(encoder_semana, 'encoder_semana.pkl')
joblib.dump(encoder_horario, 'encoder_horario.pkl')

['encoder_horario.pkl']

In [17]:
!pip install tensorflow

In [18]:
ids_amostra = dados[['conta_id', 'cliente_id']].reset_index(drop=True)
dados_sem_ids = dados.drop(columns=['conta_id', 'cliente_id'])

In [19]:
dados_numericos = dados_sem_ids.select_dtypes(include=[np.number])

In [20]:
variaveis_para_excluir = [
  'mesma_titularidade',
  'transacao_tipo_saque',
  'transacao_tipo_transferencia',
  'transacao_tipo_pagamento',
  'transacao_tipo_deposito',
  'faixa_horaria_Madrugada',
  'faixa_horaria_Manhã',
  'faixa_horaria_Tarde',
  'faixa_horaria_Noite',
  'dia_de_semana_Sabado'
]
dados_filtrados = dados_numericos.drop(columns=variaveis_para_excluir)

In [21]:
scaler = RobustScaler()
dados_normalizados = scaler.fit_transform(dados_filtrados)

In [22]:
input_dim = dados_normalizados.shape[1]
encoding_dim = 3

In [23]:
input_layer = Input(shape=(input_dim,))
encoded = Dense(42, activation='relu')(input_layer)
encoded = Dense(encoding_dim, activation='relu')(encoded)
decoded = Dense(42, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

In [24]:
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [25]:
autoencoder.fit(dados_normalizados, dados_normalizados,
                epochs=20, batch_size=256, shuffle=True, verbose=1)

Epoch 1/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 31s 4ms/step - loss: 0.0634
Epoch 2/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - loss: 0.0246
Epoch 3/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.0246
Epoch 4/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 0.0245
Epoch 5/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - loss: 0.0245
Epoch 6/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 0.0245
Epoch 7/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.0245
Epoch 8/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 0.0245
Epoch 9/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.0245
Epoch 10/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 0.0245
Epoch 11/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 0.0245
Epoch 12/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 0.0245
Epoch 13/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.0245
Epoch 14/20
6893/6893 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.0245
Epoch 15/20
689

In [26]:
reconstruido = autoencoder.predict(dados_normalizados)
erro_reconstrucao = np.mean(np.square(dados_normalizados - reconstruido), axis=1)


55143/55143 ━━━━━━━━━━━━━━━━━━━━ 67s 1ms/step


In [27]:
df_erros = ids_amostra.copy()
df_erros['erro_reconstrucao'] = erro_reconstrucao

In [28]:
encoder = Model(input_layer, encoded)
dados_latentes = encoder.predict(dados_normalizados)


55143/55143 ━━━━━━━━━━━━━━━━━━━━ 66s 1ms/step


In [29]:
kmeans_auto = KMeans(n_clusters=2, random_state=SEED)
labels_auto = kmeans_auto.fit_predict(dados_latentes)

In [30]:
df_suspeita = ids_amostra.copy()
df_suspeita['erro_reconstrucao'] = erro_reconstrucao
df_suspeita['cluster'] = labels_auto

q75 = np.quantile(erro_reconstrucao, 0.75)
q90 = np.quantile(erro_reconstrucao, 0.90)
q95 = np.quantile(erro_reconstrucao, 0.95)

condicoes = [
    df_suspeita['erro_reconstrucao'] > q95,
    df_suspeita['erro_reconstrucao'] > q90,
    df_suspeita['erro_reconstrucao'] > q75
]
valores = ['alta', 'media', 'baixa']
df_suspeita['suspeita'] = np.select(condicoes, valores, default='nenhuma')

In [31]:
dados['cluster_autoencoder'] = labels_auto

In [32]:
# Atribui os rótulos de cluster ao DataFrame
dados['cluster_autoencoder'] = labels_auto

variaveis = [
    'transacao_valor',
    'mesma_titularidade',
    'fim_de_semana',
    'transacao_tipo_pix',
    'transacao_tipo_saque',
    'transacao_tipo_transferencia',
    'transacao_tipo_pagamento',
    'transacao_tipo_deposito'
]

print("\n=== ANOVA entre clusters (Autoencoder) ===")
for var in variaveis:
    grupos = [dados[dados['cluster_autoencoder'] == i][var] for i in sorted(dados['cluster_autoencoder'].unique())]
    if all(len(grupo) > 1 for grupo in grupos):
        f_stat, p_val = f_oneway(*grupos)
        print(f"ANOVA {var}: F={f_stat:.2f}, p={p_val:.4f}")
    else:
        print(f"{var}: ignorado (cluster com poucos dados)")



=== ANOVA entre clusters (Autoencoder) ===
ANOVA transacao_valor: F=2324.98, p=0.0000
ANOVA mesma_titularidade: F=2940.68, p=0.0000
ANOVA fim_de_semana: F=3764882.68, p=0.0000
ANOVA transacao_tipo_pix: F=24141.62, p=0.0000
ANOVA transacao_tipo_saque: F=9965.70, p=0.0000
ANOVA transacao_tipo_transferencia: F=2395.78, p=0.0000
ANOVA transacao_tipo_pagamento: F=1917.80, p=0.0000
ANOVA transacao_tipo_deposito: F=767.86, p=0.0000


In [33]:
print("\n=== GLM entre clusters (Autoencoder) ===")
for var in variaveis:
    try:
        modelo = smf.glm(formula=f'{var} ~ C(cluster_autoencoder)', data=dados).fit()
        p_valor = modelo.pvalues.iloc[1]
        print(f"GLM {var}: p-valor = {p_valor:.4f}")
    except Exception as e:
        print(f"{var}: erro ao rodar GLM - {e}")



=== GLM entre clusters (Autoencoder) ===
GLM transacao_valor: p-valor = 0.0000
GLM mesma_titularidade: p-valor = 0.0000
GLM fim_de_semana: p-valor = 0.0000
GLM transacao_tipo_pix: p-valor = 0.0000
GLM transacao_tipo_saque: p-valor = 0.0000
GLM transacao_tipo_transferencia: p-valor = 0.0000
GLM transacao_tipo_pagamento: p-valor = 0.0000
GLM transacao_tipo_deposito: p-valor = 0.0000


In [34]:
perfil_comportamental =dados.groupby("conta_id").agg({
     "transacao_valor": ["mean", "std", "max", "min"],
     "mesma_titularidade": "mean",
     "fim_de_semana": "mean",
     "transacao_tipo_pix": "mean",
     "transacao_tipo_saque": "mean",
     "transacao_tipo_transferencia": "mean",
     "transacao_tipo_pagamento": "mean",
     "transacao_tipo_deposito": "mean",
     "dia_de_semana_Segunda": "mean",
     "dia_de_semana_Terca": "mean",
     "dia_de_semana_Quarta": "mean",
     "dia_de_semana_Quinta": "mean",
     "dia_de_semana_Sexta": "mean",
     "dia_de_semana_Sabado": "mean",
     "dia_de_semana_Domingo": "mean",
     "faixa_horaria_Madrugada": "mean",
     "faixa_horaria_Manhã": "mean",
     "faixa_horaria_Tarde": "mean",
     "faixa_horaria_Noite": "mean"
})

In [35]:
perfil_comportamental.columns = [''.join(col).strip() for col in perfil_comportamental.columns.values]
perfil_comportamental.reset_index(inplace=True)
perfil_comportamental

,conta_id,transacao_valormean,transacao_valorstd,transacao_valormax,transacao_valormin,mesma_titularidademean,fim_de_semanamean,transacao_tipo_pixmean,transacao_tipo_saquemean,transacao_tipo_transferenciamean,...,dia_de_semana_Tercamean,dia_de_semana_Quartamean,dia_de_semana_Quintamean,dia_de_semana_Sextamean,dia_de_semana_Sabadomean,dia_de_semana_Domingomean,faixa_horaria_Madrugadamean,faixa_horaria_Manhãmean,faixa_horaria_Tardemean,faixa_horaria_Noitemean
0,1.0518e+90,2843.470000,1209.429307,4914.79,1862.85,0.250000,0.287671,0.750000,0.000000,0.000000,...,0.142466,0.142466,0.142466,0.142466,0.142466,0.145205,0.250000,0.500000,0.250000,0.000000
1,1.18736e+28,1563.385000,792.718144,2355.56,771.21,1.000000,0.287671,0.000000,0.500000,0.500000,...,0.142466,0.142466,0.142466,0.142466,0.142466,0.145205,0.000000,1.000000,0.000000,0.000000
2,1.65269e+38,551.296667,136.311525,698.73,370.14,0.333333,0.287671,1.000000,0.000000,0.000000,...,0.142466,0.142466,0.142466,0.142466,0.142466,0.145205,0.000000,0.333333,0.666667,0.000000
3,1.67951e+75,1677.863333,1518.044709,3746.53,149.57,0.333333,0.287671,0.666667,0.333333,0.000000,...,0.142466,0.142466,0.142466,0.142466,0.142466,0.145205,0.000000,0.333333,0.666667,0.000000
4,1.91827e+65,4946.822391,1139.297403,5777.09,244.59,0.333029,0.287409,0.666971,0.000000,0.000000,...,0.142336,0.142336,0.143248,0.142336,0.142336,0.145073,0.333029,0.666058,0.000912,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,1feb42a77,4152.156667,424.218826,4604.07,3584.84,0.000000,0.287671,1.000000,0.000000,0.000000,...,0.142466,0.142466,0.142466,0.142466,0.142466,0.145205,0.000000,0.666667,0.333333,0.000000
1241,1fef6e20e,1953.470000,666.549084,3031.53,1228.01,0.375000,0.287671,0.500000,0.375000,0.000000,...,0.142466,0.142466,0.142466,0.142466,0.142466,0.145205,0.000000,0.250000,0.750000,0.000000
1242,1ff05ff6f,2455.690000,1330.717181,4195.91,571.33,0.600000,0.287671,0.400000,0.400000,0.000000,...,0.142466,0.142466,0.142466,0.142466,0.142466,0.145205,0.200000,0.600000,0.200000,0.000000
1243,1ff99ac86,407.723333,268.020233,781.06,165.21,1.000000,0.287671,0.000000,1.000000,0.000000,...,0.142466,0.142466,0.142466,0.142466,0.142466,0.145205,0.000000,0.666667,0.333333,0.000000


In [36]:
# Calcular a média e o desvio padrão para cada coluna
print(perfil_comportamental['transacao_valormean'])


0       2843.470000
1       1563.385000
2        551.296667
3       1677.863333
4       4946.822391
           ...     
1240    4152.156667
1241    1953.470000
1242    2455.690000
1243     407.723333
1244    3336.317800
Name: transacao_valormean, Length: 1245, dtype: float64


In [37]:
def gerar_perfil_cliente(dados):
    perfil = {}

    # Garantir que as colunas numéricas estão com o tipo correto
    perfil['media_valor'] = dados['transacao_valor'].mean()
    perfil['percentual_pix'] = dados['transacao_tipo_pix'].mean()
    perfil['percentual_transferencia'] = dados['transacao_tipo_transferencia'].mean()
    perfil['percentual_pagamento'] = dados['transacao_tipo_pagamento'].mean()
    perfil['percentual_saque'] = dados['transacao_tipo_saque'].mean()
    perfil['percentual_deposito'] = dados['transacao_tipo_deposito'].mean()

    perfil['percentual_fim_de_semana'] = dados['fim_de_semana'].mean()
    perfil['percentual_mesma_titularidade'] = dados['mesma_titularidade'].mean()

    faixa_horaria_cols = ['faixa_horaria_Madrugada', 'faixa_horaria_Manhã', 'faixa_horaria_Tarde', 'faixa_horaria_Noite']
    faixa_horaria_soma = dados[faixa_horaria_cols].sum()
    perfil['horario_mais_comum'] = faixa_horaria_soma.idxmax().replace('faixa_horaria_', '')

    dia_semana_cols = ['dia_de_semana_Segunda', 'dia_de_semana_Terca', 'dia_de_semana_Quarta',
                       'dia_de_semana_Quinta', 'dia_de_semana_Sexta', 'dia_de_semana_Sabado', 'dia_de_semana_Domingo']
    dia_semana_soma = dados[dia_semana_cols].sum()
    perfil['dia_semana_mais_comum'] = dia_semana_soma.idxmax().replace('dia_de_semana_', '')

    return pd.Series(perfil)


In [38]:
gerar_perfil_cliente(dados)

,0
media_valor,2533.056441
percentual_pix,0.485433
percentual_transferencia,0.098386
percentual_pagamento,0.076948
percentual_saque,0.304069
percentual_deposito,0.035164
percentual_fim_de_semana,0.287722
percentual_mesma_titularidade,0.522916
horario_mais_comum,Manhã
dia_semana_mais_comum,Domingo


In [39]:
# Mostrar o comportamento de todos os clientes
# Gera perfil de todos os clientes
perfis = dados.drop(columns='conta_id').groupby(dados['conta_id']).apply(gerar_perfil_cliente).reset_index()

# Visualiza os 5 primeiros perfis
print(perfis.head())

      conta_id  media_valor  percentual_pix  percentual_transferencia  \
0   1.0518e+90  2843.470000        0.750000                       0.0   
1  1.18736e+28  1563.385000        0.000000                       0.5   
2  1.65269e+38   551.296667        1.000000                       0.0   
3  1.67951e+75  1677.863333        0.666667                       0.0   
4  1.91827e+65  4946.822391        0.666971                       0.0   

   percentual_pagamento  percentual_saque  percentual_deposito  \
0              0.250000          0.000000                  0.0   
1              0.000000          0.500000                  0.0   
2              0.000000          0.000000                  0.0   
3              0.000000          0.333333                  0.0   
4              0.333029          0.000000                  0.0   

   percentual_fim_de_semana  percentual_mesma_titularidade horario_mais_comum  \
0                  0.287671                       0.250000              Manhã   
1 

In [40]:
resultado_clusters = ids_amostra.copy()
resultado_clusters['cluster_autoencoder'] = labels_auto
resultado_clusters

,conta_id,cliente_id,cluster_autoencoder
0,1cbe0f3f1,356,1
1,1bf29835c,596,1
2,1fd5f554a,179,1
3,1f2c8a8d7,53,1
4,1053de403,449,1
...,...,...,...
1764563,1db52b9e5,552,1
1764564,1fe6878a3,296,1
1764565,103da94f1,217,1
1764566,10d888039,26,1


In [41]:
# Verifica se 'conta_id' e 'cluster' estão no DataFrame
resultado_clusters = resultado_clusters[['conta_id', 'cluster_autoencoder']]

# Visualiza os primeiros resultados
print(resultado_clusters)

          conta_id  cluster_autoencoder
0        1cbe0f3f1                    1
1        1bf29835c                    1
2        1fd5f554a                    1
3        1f2c8a8d7                    1
4        1053de403                    1
...            ...                  ...
1764563  1db52b9e5                    1
1764564  1fe6878a3                    1
1764565  103da94f1                    1
1764566  10d888039                    1
1764567  175e5bc84                    1

[1764568 rows x 2 columns]


In [42]:
# Função para gerar o perfil calculado na hora para uma conta específica
def calcular_perfil(conta_id):
    dados_conta = dados[dados['conta_id'] == conta_id]  # usa o df original com cluster
    return gerar_perfil_cliente(dados_conta)

# Lista para armazenar os DataFrames de comparação
comparacoes = []

# Loop em todas as contas do DataFrame de perfis salvos
for conta_id in perfis['conta_id']:
    calculado = calcular_perfil(conta_id)
    salvo = perfis[perfis['conta_id'] == conta_id].drop(columns='conta_id').iloc[0]

    # Cria um DataFrame com as duas versões lado a lado
    comparacao = pd.DataFrame({
        'variavel': calculado.index,
        'Calculado_na_hora': calculado.values,
        'Salvo_em_perfis': salvo.values,
        'conta_id': conta_id
    })

    comparacoes.append(comparacao)

# Junta todos os resultados em um único DataFrame
df_comparado = pd.concat(comparacoes, ignore_index=True)

# Visualiza os primeiros resultados
print(df_comparado.head(20))


                         variavel Calculado_na_hora Salvo_em_perfis  \
0                     media_valor           2843.47         2843.47   
1                  percentual_pix              0.75            0.75   
2        percentual_transferencia               0.0             0.0   
3            percentual_pagamento              0.25            0.25   
4                percentual_saque               0.0             0.0   
5             percentual_deposito               0.0             0.0   
6        percentual_fim_de_semana          0.287671        0.287671   
7   percentual_mesma_titularidade              0.25            0.25   
8              horario_mais_comum             Manhã           Manhã   
9           dia_semana_mais_comum           Domingo         Domingo   
10                    media_valor          1563.385        1563.385   
11                 percentual_pix               0.0             0.0   
12       percentual_transferencia               0.5             0.5   
13    

In [43]:
np.unique(labels_auto, return_counts=True)

(array([0, 1], dtype=int32), array([1107569,  656999]))

In [44]:
# Adiciona os rótulos de cluster ao DataFrame completo, se ainda não estiver
dados['cluster_autoencoder'] = labels_auto

# Lista das variáveis comportamentais principais
variaveis_comportamentais = [
    'transacao_valor',
    'mesma_titularidade',
    'fim_de_semana',
    'transacao_tipo_pix',
    'transacao_tipo_saque',
    'transacao_tipo_transferencia',
    'transacao_tipo_pagamento',
    'transacao_tipo_deposito',
    'dia_de_semana_Segunda',
    'dia_de_semana_Terca',
    'dia_de_semana_Quarta',
    'dia_de_semana_Quinta',
    'dia_de_semana_Sexta',
    'dia_de_semana_Sabado',
    'dia_de_semana_Domingo',
    'faixa_horaria_Madrugada',
    'faixa_horaria_Manhã',
    'faixa_horaria_Tarde',
    'faixa_horaria_Noite'
]

# Calcula a média de cada variável por cluster
perfil_clusters = dados.groupby('cluster_autoencoder')[variaveis_comportamentais].mean().T

# Adiciona uma coluna com a diferença absoluta entre os dois clusters
perfil_clusters['diferença_absoluta'] = abs(perfil_clusters[0] - perfil_clusters[1])

# Ordena pela maior diferença
perfil_clusters_ordenado = perfil_clusters.sort_values(by='diferença_absoluta', ascending=False)


In [45]:
# Exibe os perfis comportamentais ordenados pela diferença entre clusters
print(perfil_clusters_ordenado)

# Ou para visualização interativa no Colab
perfil_clusters_ordenado.head(20)  # mostra os 20 mais diferentes


cluster_autoencoder                     0            1  diferença_absoluta
transacao_valor               2591.622666  2434.325492          157.297173
fim_de_semana                    0.000020     0.772730            0.772710
dia_de_semana_Domingo            0.000000     0.390165            0.390165
dia_de_semana_Sabado             0.000020     0.382565            0.382545
dia_de_semana_Quarta             0.226983     0.000000            0.226983
dia_de_semana_Quinta             0.226952     0.000000            0.226952
dia_de_semana_Sexta              0.226947     0.000000            0.226947
dia_de_semana_Terca              0.226946     0.000000            0.226946
dia_de_semana_Segunda            0.092153     0.227270            0.135117
transacao_tipo_pix               0.440714     0.560821            0.120107
transacao_tipo_saque             0.330620     0.259309            0.071310
mesma_titularidade               0.538607     0.496463            0.042143
transacao_tipo_transferen

cluster_autoencoder,0,1,diferença_absoluta
transacao_valor,2591.622666,2434.325492,157.297173
fim_de_semana,0.000020,0.772730,0.772710
dia_de_semana_Domingo,0.000000,0.390165,0.390165
dia_de_semana_Sabado,0.000020,0.382565,0.382545
dia_de_semana_Quarta,0.226983,0.000000,0.226983
dia_de_semana_Quinta,0.226952,0.000000,0.226952
dia_de_semana_Sexta,0.226947,0.000000,0.226947
dia_de_semana_Terca,0.226946,0.000000,0.226946
dia_de_semana_Segunda,0.092153,0.227270,0.135117
transacao_tipo_pix,0.440714,0.560821,0.120107


In [46]:
autoencoder.save('modelo_autoencoder.keras')

In [47]:
encoder.save('modelo_encoder.keras')

In [48]:
joblib.dump(kmeans_auto, 'kmeans_auto.pkl')

['kmeans_auto.pkl']

In [49]:
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [50]:
perfis.to_csv('perfil_comportamentais.csv', index=False)

In [51]:
df_suspeita.to_csv('transacoes_com_suspeita.csv', index=False)

In [52]:
joblib.dump(dados_filtrados.columns.tolist(), 'colunas_scaler.pkl')

['colunas_scaler.pkl']